In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import urllib.parse
from copy import copy
import requests
import re
import json
import urllib.request

## Scrape event information for a driver

In [2]:
def events_for_a_driver(my_url):
    
    # Load races on url
    raw_html = requests.get(my_url).content
    soup_doc = BeautifulSoup(raw_html, "html.parser")
    events = soup_doc.find_all("div", class_="profile-start-line d-flex justify-content-start w-100 lh-120 py-1 flex-wrap")
    races = []
    
    # Find all WRC races
    events = [e for e in events if e.span.text == ' WRC']

    # List out all informations
    
    for e in events:
        race = {}
        race['Driver'] = soup_doc.find("h4").a.text.strip()
        race['Name'] = e.find("a", title="Show entry info and stats").text
        URL = e.find("a", title="Show entry info and stats")['href']
        race['URL'] = URL
    # Check the URL
        print(URL)
        id1 = re.findall("/entryinfo/(\d+)",URL)[0]
        id2 = re.findall("/entryinfo/.+/(\d+)",URL)[0]
        Year = re.findall("/entryinfo/.+-(\d\d\d\d)/\d+/",URL)[0]
        race['Year'] = Year
        
        
        try:
            rank = e.find("span", class_="fs-09 font-weight-normal text-secondary").text
            ra = re.findall("(\d).",rank)[0]
            race['Race_Result'] = ra
            race['DNG_Reason'] = "Null"
        except:
            ra = e.find("div", class_="profile-start-ret text-danger font-weight-bold")
            race['Race_Result'] = "DNF"
            race['DNG_Reason'] = ra
        
        race['Php_url'] = "https://www.ewrc-results.com/inc/json_driverinfo.php?event="+id1+"&entry="+id2+"&sct=&cat=&lg=en"
        
        print("Php found" + race['Php_url'])
        
        x = urllib.request.urlopen(race['Php_url'])
        raw_data = x.read()
        encoding = x.info().get_content_charset('utf8')  # JSON default
        ssresult = json.loads(raw_data.decode(encoding))
        print(ssresult, len(ssresult['stages'][0][1]))

        
        for idx, pos in enumerate(ssresult['pos'][0][1]):
            
            stage = race.copy()
            
            stage['p_overall'] = pos      
            stage['stage'] = ssresult['stages'][0][1][idx]
            stage['ssname'] = ssresult['sslist'][idx]
            races.append(stage)
            # print(ssresult['pos'][0][1])
        
        
        print("Php info scraped.")
        
        
    return races



In [3]:
races = events_for_a_driver('https://www.ewrc-results.com/profile/3109-sebastien-ogier/')

/entryinfo/77787-rallye-automobile-monte-carlo-2023/3560663/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=77787&entry=3560663&sct=&cat=&lg=en
{'pos': [['Overall', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [1, 1, 1, 1, 1, 2, 1, 3, 3, 1, 2, 4, 4, 4, 1, 2, 1, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'colors': ['#e2431e', '#f1ca3a']} 18
Php info scraped.
/entryinfo/73369-forum8-rally-japan-2022/3507849/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=73369&entry=3507849&sct=&cat=&lg=en
{'pos': [['Overall', [1, 33, 32, 14, 10, 7, 6, 6, 5, 5, 5, 5, 5, 5, 4, 4]]], 'stages': [['Stage results', [1, 33, 2, 3, 6, 3, 1, 2, 1, 1, 3, 2, 2, 1, 2, 7]]], 'sslist': ['SS1', 'SS2', 'SS4', 'SS5', 'SS6', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e',

{'pos': [['Overall', [5, 3, 3, 3, 3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1]]], 'stages': [['Stage results', [5, 3, 1, 3, 3, 1, 1, 1, 1, 2, 4, 1, 7, 4, 1, 1, 3, 3, 3, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20'], 'colors': ['#e2431e', '#f1ca3a']} 20
Php info scraped.
/entryinfo/69715-arctic-rally-finland-powered-by-capitalbox-2021/2918694/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=69715&entry=2918694&sct=&cat=&lg=en
{'pos': [['Overall', [9, 9, 9, 8, 8, 7, 6, 22, 20, 20]]], 'stages': [['Stage results', [9, 10, 6, 3, 3, 8, 5, 47, 8, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10'], 'colors': ['#e2431e', '#f1ca3a']} 10
Php info scraped.
/entryinfo/41079-rallye-automobile-monte-carlo-2021/2872591/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=41079&entry=2872591&sct=&cat=&lg=en


{'pos': [['Overall', [4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]]], 'stages': [['Stage results', [4, 2, 4, 4, 3, 4, 5, 5, 5, 1, 3, 4, 4, 1, 2, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16'], 'colors': ['#e2431e', '#f1ca3a']} 16
Php info scraped.
/entryinfo/54465-xion-rally-argentina-2019/2197562/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=54465&entry=2197562&sct=&cat=&lg=en
{'pos': [['Overall', [2, 7, 5, 3, 4, 3, 2, 3, 3, 3, 6, 5, 3, 3, 3, 4, 3]]], 'stages': [['Stage results', [2, 8, 3, 2, 5, 3, 2, 5, 5, 8, 15, 1, 1, 3, 3, 5, 1]]], 'sslist': ['SS1', 'SS2', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'colors': ['#e2431e', '#f1ca3a']} 17
Php info scraped.
/entryinfo/54464-corsica-linea-tour-de-corse-2019/2121688/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=54464&entry=2121688&s

{'pos': [['Overall', [3, 3, 6, 5, 5, 5, 6, 5, 5, 5, 4, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [3, 4, 7, 8, 7, 2, 6, 4, 2, 1, 2, 2, 1, 1, 1, 1, 6, 4, 2, 2, 4, 7]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22'], 'colors': ['#e2431e', '#f1ca3a']} 22
Php info scraped.
/entryinfo/42875-rally-sweden-2018/1700198/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=42875&entry=1700198&sct=&cat=&lg=en
{'pos': [['Overall', [9, 10, 11, 11, 11, 11, 11, 12, 12, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10]]], 'stages': [['Stage results', [9, 10, 12, 11, 12, 11, 22, 20, 10, 9, 3, 12, 9, 10, 4, 7, 6, 10, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 19
Php info scraped.
/entryinfo/42870-

{'pos': [['Overall', [6, 8, 8, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2]]], 'stages': [['Stage results', [6, 8, 8, 2, 3, 3, 4, 1, 1, 1, 1, 4, 2, 1, 2, 2, 1, 2, 2, 3, 3, 1, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23'], 'colors': ['#e2431e', '#f1ca3a']} 23
Php info scraped.
/entryinfo/27499-dayinsure-wales-rally-gb-2016/1281092/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=27499&entry=1281092&sct=&cat=&lg=en
{'pos': [['Overall', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 4, 2, 2, 3, 1, 2, 2, 2, 2, 4, 6, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22'], 'colors': ['#e2431e', '#f1ca3a']}

...

{'pos': [['Overall', [1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [1, 3, 1, 7, 2, 1, 3, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 7, 5, 16, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23', 'SS24'], 'colors': ['#e2431e', '#f1ca3a']} 22
Php info scraped.
/entryinfo/13896-rally-italia-sardegna-2014/562159/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=13896&entry=562159&sct=&cat=&lg=en
{'pos': [['Overall', [2, 9, 10, 8, 6, 4, 4, 3, 2, 2, 2, 2, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [2, 9, 12, 7, 5, 2, 2, 1, 2, 2, 1, 1, 1, 8, 3, 3, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17'], 'colors': ['#e2431e', '#f1ca3a']} 17
Php info scraped.
/entryinfo/13895-xion-rally-argentina-2014/537035/
Php foundht

{'pos': [['Overall', [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [1, 3, 3, 4, 2, 2, 2, 1, 1, 1, 1, 9, 2, 3, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15'], 'colors': ['#e2431e', '#f1ca3a']} 15
Php info scraped.
/entryinfo/6533-rally-guanajuato-mexico-2013/265236/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=6533&entry=265236&sct=&cat=&lg=en
{'pos': [['Overall', [4, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [4, 1, 1, 3, 2, 1, 1, 1, 4, 4, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23'], 'colors': ['#e2431e', '#f1ca3a']} 23
Php info scraped.
/entryinfo/6532-rally-sweden-2013/244687/
Php foundhttps://www.ewrc-results.com/i

{'pos': [['Overall', [12, 6, 6, 4, 4, 6, 5, 6, 6]]], 'stages': [['Stage results', [12, 3, 8, 6, 7, 7, 4, 10, 8]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'colors': ['#e2431e', '#f1ca3a']} 9
Php info scraped.
/entryinfo/2373-wales-rally-gb-2011/132812/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=2373&entry=132812&sct=&cat=&lg=en
{'pos': [['Overall', [73, 73, 73, 70, 66, 65, 60, 52, 48, 40, 27, 20, 20, 17, 15, 15, 15, 14, 13, 13, 13, 12, 11]]], 'stages': [['Stage results', [73, 73, 65, 8, 7, 8, 5, 6, 8, 6, 5, 4, 3, 3, 3, 4, 3, 5, 2, 1, 2, 1, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23'], 'colors': ['#e2431e', '#f1ca3a']} 23
Php info scraped.
/entryinfo/2372-rallyracc-catalunya-costa-daurada-2011/130723/
Php foundht

{'pos': [['Overall', [2, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 6, 6, 6, 6, 6]]], 'stages': [['Stage results', [2, 3, 4, 3, 3, 3, 3, 4, 3, 4, 4, 4, 1, 7, 38, 45, 5, 6, 5]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS20'], 'colors': ['#e2431e', '#f1ca3a']} 19
Php info scraped.
/entryinfo/582-rally-japan-2010/78445/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=582&entry=78445&sct=&cat=&lg=en
{'pos': [['Overall', [1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2, 2, 3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [1, 1, 2, 5, 6, 3, 4, 8, 6, 2, 5, 4, 2, 4, 2, 6, 2, 3, 2, 1, 2, 1, 2, 6, 1, 7]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23', 'SS24', 'SS25', 'SS26'], 'colors': ['#e2431e', '#f1ca3a']} 26
Php info scrape

{'pos': [['Overall', [6, 9, 8, 6, 7, 7, 6, 6, 6, 4, 4, 4, 4, 4, 4]]], 'stages': [['Stage results', [6, 9, 9, 6, 7, 6, 6, 5, 3, 3, 11, 7, 4, 1, 4]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18'], 'colors': ['#e2431e', '#f1ca3a']} 15
Php info scraped.
/entryinfo/394-acropolis-rally-of-greece-2009/32836/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=394&entry=32836&sct=&cat=&lg=en
{'pos': [['Overall', [9, 7, 10, 7, 6, 4, 4, 3, 3, 2, 2, 2, 2, 2, 2]]], 'stages': [['Stage results', [7, 9, 12, 1, 8, 6, 6, 5, 8, 4, 5, 6, 6, 6, 7]]], 'sslist': ['SS1', 'SS2', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS17'], 'colors': ['#e2431e', '#f1ca3a']} 15
Php info scraped.
/entryinfo/393-rally-ditalia-sardegna-2009/32506/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=393&entry=32506&sct=&cat=&lg=en
{'pos': [['Overal

In [5]:
df = pd.DataFrame(races)
df

Driver                           Name  \
0     Sébastien Ogier  Rallye Automobile Monte-Carlo   
1     Sébastien Ogier  Rallye Automobile Monte-Carlo   
2     Sébastien Ogier  Rallye Automobile Monte-Carlo   
3     Sébastien Ogier  Rallye Automobile Monte-Carlo   
4     Sébastien Ogier  Rallye Automobile Monte-Carlo   
...               ...                            ...   
3063  Sébastien Ogier        Neste Oil Rally Finland   
3064  Sébastien Ogier        Neste Oil Rally Finland   
3065  Sébastien Ogier        Neste Oil Rally Finland   
3066  Sébastien Ogier        Neste Oil Rally Finland   
3067  Sébastien Ogier        Neste Oil Rally Finland   

                                                    URL  Year Race_Result  \
0     /entryinfo/77787-rallye-automobile-monte-carlo...  2023           1   
1     /entryinfo/77787-rallye-automobile-monte-carlo...  2023           1   
2     /entryinfo/77787-rallye-automobile-monte-carlo...  2023           1   
3     /entryinfo/77787-rallye-automobile-monte-carlo...  2023           1   
4     /entryinfo/77787-rallye-automobile-monte-carlo...  2023           1   
...                                                 ...   ...         ...   
3063  /entryinfo/157-neste-oil-rally-finland-2008/17...  2008           3   
3064  /entryinfo/157-neste-oil-rally-finland-2008/17...  2008           3   
3065  /entryinfo/157-neste-oil-rally-finland-2008/17...  2008           3   
3066  /entryinfo/157-neste-oil-rally-finland-2008/17...  2008           3   
3067  /entryinfo/157-neste-oil-rally-finland-2008/17...  2008           3   

     DNG_Reason                                            Php_url  p_overall  \
0          Null  https://www.ewrc-results.com/inc/json_driverin...          1   
1          Null  https://www.ewrc-results.com/inc/json_driverin...          1   
2          Null  https://www.ewrc-results.com/inc/json_driverin...          1   
3          Null  https://www.ewrc-results.com/inc/json_driverin...          1   
4          Null  https://www.ewrc-results.com/inc/json_driverin...          1   
...         ...                                                ...        ...   
3063       Null  https://www.ewrc-results.com/inc/json_driverin...         27   
3064       Null  https://www.ewrc-results.com/inc/json_driverin...         26   
3065       Null  https://www.ewrc-results.com/inc/json_driverin...         35   
3066       Null  https://www.ewrc-results.com/inc/json_driverin...         36   
3067       Null  https://www.ewrc-results.com/inc/json_driverin...         35   

      stage ssname  
0         1    SS1  
1         1    SS2  
2         1    SS3  
3         1    SS4  
4         1    SS5  
...     ...    ...  
3063     22   SS20  
3064     20   SS21  
3065     71   SS22  
3066     53   SS23  
3067     52   SS24  

[3068 rows x 10 columns]

In [6]:
df.to_csv('Sébastien_Ogier.csv')